In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tslearn.clustering import silhouette_score
import matplotlib as mpl

In [ ]:
with open('./pickles/kmeans_total_3.pickle', 'rb') as handle:
    kmeans_total3 = pickle.load(handle)

with open('./pickles/kmeans_total_4.pickle', 'rb') as handle:
    kmeans_total4 = pickle.load(handle)

with open('./pickles/kmeans_total_6.pickle', 'rb') as handle:
    kmeans_total6 = pickle.load(handle)

with open('./pickles/newSeries_total_3.pickle', 'rb') as handle:
    newSeries_total3 = pickle.load(handle)

with open('./pickles/newSeries_total_4.pickle', 'rb') as handle:
    newSeries_total4 = pickle.load(handle)

with open('./pickles/newSeries_total_6.pickle', 'rb') as handle:
    newSeries_total6 = pickle.load(handle)

with open('./pickles/rese_total_3.pickle', 'rb') as handle:
    rese_total_3 = pickle.load(handle)

with open('./pickles/rese_total_4.pickle', 'rb') as handle:
    rese_total_4 = pickle.load(handle)

with open('./pickles/rese_total_6.pickle', 'rb') as handle:
    rese_total_6 = pickle.load(handle)

with open('./pickles/newResa3.pickle', 'rb') as handle:
    newResa3 = pickle.load(handle)

with open('./pickles/newResa4.pickle', 'rb') as handle:
    newResa4 = pickle.load(handle)

with open('./pickles/newResa6.pickle', 'rb') as handle:
    newResa6 = pickle.load(handle)

In [ ]:
# Numero di cluster 

num_cluster = 2

# Rese pulite

newResa3 = newResa3[(newResa3<=11000) & (newResa3 >= 4000)]

newResa4 = newResa4[(newResa4<=11000) & (newResa4 >= 4000)]

newResa6 = newResa6[(newResa6<=11000) & (newResa6 >= 4000)]

# Andiamo a calcolare una nuova metrica di coesione